In [ ]:
train_labels

In [2]:
# %load '/home/dnnlab/vaisakh/anaconda3/Code/LOG/dnnDynamicTF.py'
"""
Created on Sun Dec 1 2:23:09 2016

@author: Vaisakh 
email: vaisakhs.shaj@gmail.com
"""

############## Import Libraries Here###################

from __future__ import print_function
import numpy as np
import tensorflow as tf
import pickle


class FLAGS:
    def __init__(self):
        self.randomInit = True
        self.b = 0
        
############# Set Parameters here #####################
FLAG =FLAGS()
FLAG.randomInit = False
checkPointPath='/home/dnnlab/vaisakh/familyClassification/Model/45layerMTL.ckpt'
checkPointPath2='/home/dnnlab/vaisakh/familyClassification/Model/45layerMTL.ckpt'
checkPointFolder='/home/dnnlab/vaisakh/familyClassification/Model'
#input_size = 1270
input_size=957
num_labels_1 = 2
num_labels_2 = 21

batch_size_1 =5000
batch_size_2 =10000
#H=[ 1525, 1018] #hidden layer size
H1=[ 1100, 760] #hidden layer size
H2= [ 1100,760,480 ]       
#print(num_labels)
lam=0
num_steps = 10001
learn_rate=0.01





############ Load Data and preprocess

pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/xMillionTrainL1-2001.pickle","rb")
train_dataset1=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/yMillionTrainL1-2001.pickle","rb")
train_labels1=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/xMillionTrainL1-21.pickle","rb")
train_dataset2=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/yMillionTrainL1-21.pickle","rb")
train_labels2=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/xMillionCrossL1-2001.pickle","rb")
valid_dataset=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/yMillionCrossL1-2001.pickle","rb")
valid_labels=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/xMillionCrossL1-21.pickle","rb")
test_dataset=pickle.load(pickle_in)
pickle_in=open("/home/dnnlab/vaisakh/familyClassification/Data/yMillionCrossL1-21.pickle","rb")
test_labels=pickle.load(pickle_in)

print('Training set 1', train_dataset1.shape, train_labels1.shape)
print('Training set 2', train_dataset2.shape, train_labels2.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)



##=================Preprocessing


train_labels1=(np.arange(num_labels_1)==train_labels1[:,None].astype(np.float32))
train_labels2=(np.arange(num_labels_2)==train_labels2[:,None].astype(np.float32))
valid_labels=(np.arange(num_labels_1)==valid_labels[:,None].astype(np.float32))
test_labels=(np.arange(num_labels_2)==test_labels[:,None].astype(np.float32))



############ Training ################################################################l
def predictTask1(X,w1,w2,w3,b1,b2,b3):
     layer2= tf.matmul(X, w1) + b1
     layer2out = tf.nn.relu(layer2)
     layer3 = tf.matmul(layer2out, w2) + b2
     layer3out = tf.nn.relu(layer3)
     layer4 = tf.matmul(layer3out, w3) + b3
     layer4out = tf.nn.softmax(layer4)
     return layer4out
    
def predictTask2(X,w1,w2,w3,w4,b1,b2,b3,b4):
     layer2= tf.matmul(X, w1) + b1
     layer2out = tf.nn.relu(layer2)
     layer3 = tf.matmul(layer2out, w2) + b2
     layer3out = tf.nn.relu(layer3)
     layer4 = tf.matmul(layer3out, w3) + b3
     layer4out = tf.nn.relu(layer4)
     layer5 = tf.matmul(layer4out, w4) + b4
     layer5out = tf.nn.softmax(layer5)
     return layer5out

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def accuracyTest(predictions, labels):
  acti=np.max(predictions, 1)
  yPr=np.argmax(predictions, 1)
  yAc=np.argmax(labels, 1)
  print(confusion_matrix(yAc,yPr))
  print(accuracy_score(yAc,yPr))
  return 1
###=====================Defining Graphs========================######
graph = tf.Graph()
with graph.as_default():
  #with tf.device('/gpu:1'):
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    tf_train_dataset1 = tf.placeholder(tf.float32,
                                  shape=(batch_size_1, input_size))
    tf_train_labels1 = tf.placeholder(tf.float32, shape=(batch_size_1, num_labels_1))
    tf_train_dataset2 = tf.placeholder(tf.float32,
                                  shape=(batch_size_2, input_size))
    tf_train_labels2 = tf.placeholder(tf.float32, shape=(batch_size_2, num_labels_2))
    tf_valid_dataset = tf.constant(valid_dataset,dtype=tf.float32)
    tf_test_dataset = tf.constant(test_dataset,dtype=tf.float32)
  
  

# Variables , here the shared weight being the weight1 .
    weights1 = tf.Variable(
    tf.truncated_normal([input_size, H1[0]]),name="weights1")
    biases1 = tf.Variable(tf.zeros([H1[0]]),name="biases1")
    weights2_task1 = tf.Variable(
        tf.truncated_normal([H1[0] , H1[1]]),name="weights21")
    biases2_task1 = tf.Variable(tf.zeros([H1[1]]),name="biases21")
    weights2_task2 = tf.Variable(
        tf.truncated_normal([H2[0] , H2[1]]),name="weights22")
    biases2_task2 = tf.Variable(tf.zeros([H2[1]]),name="biases22")
    weights3_task1 = tf.Variable(
        tf.truncated_normal([H1[1] , num_labels_1]),name="weights31")
    biases3_task1 = tf.Variable(tf.zeros([num_labels_1]),name="biases31")
    weights3_task2 = tf.Variable(
        tf.truncated_normal([H2[1] , H2[2]]),name="weights32")
    biases3_task2 = tf.Variable(tf.zeros([H2[2]]),name="biases32")
    weights4_task2 = tf.Variable(
        tf.truncated_normal([H2[2] , num_labels_2]),name="weights42")
    biases4_task2 = tf.Variable(tf.zeros([num_labels_2]),name="biases42")
    
  
  
# Training computation for the two tasks.
    #Task 1
    X=tf_train_dataset1
    layer2 = tf.matmul(X, weights1) + biases1
    layer2out = tf.nn.relu(layer2)
    layer2out = tf.nn.dropout(layer2out,0.5)
    layer31 = tf.matmul(layer2out, weights2_task1) + biases2_task1
    layer31out = tf.nn.relu(layer31)
    layer31out = tf.nn.dropout(layer31out,0.5)
    layer41 = tf.matmul(layer31out, weights3_task1) + biases3_task1
    output1 = tf.nn.softmax(layer41)
    
    #Task 2
    X=tf_train_dataset2
    layer2 = tf.matmul(X, weights1) + biases1
    layer2out = tf.nn.relu(layer2)
    layer2out = tf.nn.dropout(layer2out,0.5)
    layer32 = tf.matmul(layer2out, weights2_task2) + biases2_task2
    layer32out = tf.nn.relu(layer32)
    layer32out = tf.nn.dropout(layer32out,0.5)
    layer42 = tf.matmul(layer32out, weights3_task2) + biases3_task2
    layer42out = tf.nn.relu(layer42)
    layer42out = tf.nn.dropout(layer42out,0.5)
    layer52 = tf.matmul(layer42out, weights4_task2) + biases4_task2
    output2 = tf.nn.softmax(layer52)

    loss1 = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(layer41, tf_train_labels1)) 
    loss2 = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(layer52, tf_train_labels2))

# Optimizers for the two tasks.
    optimizer1 = tf.train.AdamOptimizer(learn_rate).minimize(loss1)
    optimizer2 = tf.train.AdamOptimizer(learn_rate).minimize(loss2)

# Predictions for the training, validation, and test data.
    train_prediction1 = output1
    train_prediction2 = output2
    X=tf_valid_dataset
    valid_prediction = predictTask1(X,weights1,weights2_task1,weights3_task1,biases1,biases2_task1,biases3_task1)
    X=tf_test_dataset
    test_prediction = predictTask2(X,weights1,weights2_task2,weights3_task2,weights4_task2,biases1,biases2_task2,
                                   biases3_task2,biases4_task2)


######==========Running Graphs========================================


with tf.Session(graph=graph) as session:
     #with tf.device("/gpu:0"):
        saver=tf.train.Saver(var_list={"biases1": biases1, "weights1": weights1, "biases21": biases2_task1, 
                                   "weights21": weights2_task1 ,"biases22": biases2_task2, 
                                   "weights22": weights2_task2 ,"biases31": biases3_task1, 
                                   "weights31": weights3_task1 ,"biases32": biases3_task2, 
                                   "weights32": weights3_task2 ,"biases42": biases4_task2, 
                                   "weights42": weights4_task2 })
        if FLAG.randomInit:
            tf.initialize_all_variables().run()
            print("Initialized")
        else:
            tf.initialize_all_variables().run()
            saver.restore(session,checkPointPath)
        step_1=0
        step_2=0
        for step in range(num_steps):
            if step==0 or np.random.rand() < 0.5:
            # Pick an offset within the training data 1, which has been randomized.
            # Note: we could use better randomization across epochs.
                offset_1 = (step_1 * batch_size_1) % (train_labels1.shape[0] - batch_size_1)
                step_1=step_1+1
            # Generate a minibatch1.
                batch_data_1 = train_dataset1[offset_1:(offset_1 + batch_size_1), :]
                batch_labels_1 = train_labels1[offset_1:(offset_1 + batch_size_1), :]
            # Prepare a dictionary telling the session where to feed the minibatch1.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
                feed_dict_1 = {tf_train_dataset1 : batch_data_1, tf_train_labels1 : batch_labels_1}
                _, l1, predictions1 = session.run(
                [optimizer1, loss1, train_prediction1], feed_dict=feed_dict_1)
            if step==0 or np.random.rand() >= 0.5:
            # Pick an offset within the training data1, which has been randomized.
            # Note: we could use better randomization across epochs.
                offset_2 = (step_2 * batch_size_2) % (train_labels2.shape[0] - batch_size_2)
                step_2=step_2+1
            # Generate a minibatch2.
                batch_data_2 = train_dataset2[offset_2:(offset_2 + batch_size_2), :]
                batch_labels_2 = train_labels2[offset_2:(offset_2 + batch_size_2), :]
            # Prepare a dictionary telling the session where to feed the minibatch2.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
                feed_dict_2 = {tf_train_dataset2 : batch_data_2, tf_train_labels2 : batch_labels_2}
                _, l2, predictions2 = session.run(
                [optimizer2, loss2, train_prediction2], feed_dict=feed_dict_2)
           
            
            if (step % 300 == 0):
                print("Minibatch loss task1 at step %d: %f" % (step, l1))
                print("Minibatch loss task2 at step %d: %f" % (step, l2))
                print("Minibatch accuracy task 1: %.1f%%" % accuracy(predictions1, batch_labels_1))
                print("Minibatch accuracy task 2: %.1f%%" % accuracy(predictions2, batch_labels_2))
                print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
                print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

            if(step==num_steps-1):
                save_path=saver.save(session,checkPointPath2)
        print(session.run(weights1))   
        session.close()









Training set 1 (57274, 957) (57274,)
Training set 2 (117280, 957) (117280,)
Validation set (6363, 957) (6363,)
Test set (2449, 957) (2449,)
Minibatch loss task1 at step 0: 0.164956
Minibatch loss task2 at step 0: 0.272763
Minibatch accuracy task 1: 93.3%
Minibatch accuracy task 2: 92.4%
Validation accuracy: 92.4%
Test accuracy: 81.6%
Minibatch loss task1 at step 300: 0.164587
Minibatch loss task2 at step 300: 0.242000
Minibatch accuracy task 1: 93.7%
Minibatch accuracy task 2: 93.3%
Validation accuracy: 92.7%
Test accuracy: 81.4%
Minibatch loss task1 at step 600: 0.151284
Minibatch loss task2 at step 600: 0.235301
Minibatch accuracy task 1: 93.3%
Minibatch accuracy task 2: 93.2%
Validation accuracy: 92.7%
Test accuracy: 81.9%
Minibatch loss task1 at step 900: 0.147333
Minibatch loss task2 at step 900: 0.233587
Minibatch accuracy task 1: 93.6%
Minibatch accuracy task 2: 93.4%
Validation accuracy: 92.7%
Test accuracy: 81.9%
Minibatch loss task1 at step 1200: 0.146597
Minibatch loss task2